In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from datetime import timedelta, datetime

In [10]:
class Model(object):
    def __init__(self, country, data_dir, start_date, loss):
        self.country = country
        self.data_dir = data_dir
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        self.loss = loss

    def _load_data(self):
        df = pd.read_csv(self.data_dir)
        df['date'] = pd.to_datetime(df['date'])
        df = df[df['date'] >= self.start_date].reset_index()
        df['R'] = df['deaths'] + df['recovered']
        df['I'] = df['confirmed'] - df['R']
        return df

    def _extend_index(self, index, new_size):
        values = index.values
        current = datetime.strptime(index[-1], '%m/%d/%y')
        while len(values) < new_size:
            current = current + timedelta(days=1)
            values = np.append(values, datetime.strftime(current, '%m/%d/%y'))
        return values

    def _predict(self, beta, alpha, gamma, data, recovered):
        predict_range = 150
        new_index = self._extend_index(data.index, predict_range)
        size = len(new_index)
        def SEIR(t, y):
            S = y[0]
            E = y[1]
            I = y[2]
            R = y[3]
            return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
        extended_actual = np.concatenate((data.values, [None] * (size - len(data.values))))
        extended_recovered = np.concatenate((recovered.values, [None] * (size - len(recovered.values))))
        return new_index, extended_actual, extended_recovered, solve_ivp(SEIR, [0, size], [S_0,I_0,R_0], t_eval=np.arange(0, size, 1))

    def train(self):
        data = self._load_confirmed()
        # print(data)
        recovered = self.load_recovered(self.country)
        optimal = minimize(
            loss,
            [0.001, 0.001],
            args=(data, recovered),
            method='L-BFGS-B',
            bounds=[(0.00000001, 0.4), (0.00000001, 0.4)]
        )
        print(optimal)
        beta, gamma = optimal.x
        new_index, extended_actual, extended_recovered, prediction = self.predict(beta, gamma, data, recovered, self.country)
        df = pd.DataFrame({
            'Confirmed': extended_actual, 
            'Recovered': extended_recovered, 
            'S': prediction.y[0], 
            'I': prediction.y[1], 
            'R': prediction.y[2]}, 
        index=new_index)
        fig, ax = plt.subplots(figsize=(15, 10))
        ax.set_title(self.country)
        df.plot(ax=ax)
        print(f"country={self.country}, beta={beta:.8f}, gamma={gamma:.8f}, r_0:{(beta/gamma):.8f}")
        fig.savefig(f"{self.country}.png")

        
def loss(point, data, recovered):
    size = len(data)
    beta, gamma = point
    def SIR(t, y):
        S = y[0]
        I = y[1]
        R = y[2]
        return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
    solution = solve_ivp(SIR, [0, size], [S_0,I_0,R_0], t_eval=np.arange(0, size, 1), vectorized=True)
    l1 = np.sqrt(np.mean((solution.y[1] - data)**2))
    l2 = np.sqrt(np.mean((solution.y[2] - recovered)**2))
    alpha = 0.1
    return alpha * l1 + (1 - alpha) * l2

In [11]:
model_Italy = Model('Italy', 'output/italy.csv', '2020-01-31', loss)
model_Italy.train()

9         2
10        2
11        2
12        2
13        2
14        2
15        2
16        3
17        3
18        3
19        3
20        3
21        3
22        3
23        3
24        3
25        3
26        3
27        3
28        3
29        3
30       20
31       62
32      155
33      229
34      322
35      453
36      655
37      888
38     1128
39     1694
40     2036
41     2502
42     3089
43     3858
44     4636
45     5883
46     7375
47     9172
48    10149
49    12462
50    12462
51    17660
52    21157
53    24747
54    27980
55    31506
56    35713
57    41035
Name: confirmed, dtype: int64


In [ ]:
def seir_model_with_soc_dist(init_vals, params, t):
    S_0, E_0, I_0, R_0 = init_vals
    S, E, I, R = [S_0], [E_0], [I_0], [R_0]
    alpha, beta, gamma, rho = params
    dt = t[1] - t[0]
    for _ in t[1:]:
        next_S = S[-1] - (rho*beta*S[-1]*I[-1])*dt
        next_E = E[-1] + (rho*beta*S[-1]*I[-1] - alpha*E[-1])*dt
        next_I = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
        next_R = R[-1] + (gamma*I[-1])*dt
        S.append(next_S)
        E.append(next_E)
        I.append(next_I)
        R.append(next_R)
    return np.stack([S, E, I, R]).T